## Capítulo 6

Acessar dados é um primeiro passo necessário para usar a maior parte das ferramentas. A entrada e a saída de dados geralmente se enquadram em algumas categorias principais: leitura de arquivos-textos e outros formatos mais eficientes em disco, carga de dados de bancos de dados e interação com fontes de dados da rede, como API's web.

### 6.1 Lendo e escrevendo dados em formato-texto

O pandas tem uma série de funções para ler dados tabulares na forma de um objeto DataFrame. A tabela 6.1 sintetiza algumas delas, embora 'read_csv' e 'read_table' provavelmente sejam aquelas que usará com mais frequência.

###### Tabela 6.1 -> Funções de parsing do pandas

Função

* read_csv => Carrega dados delimitados de um arquivo, um URL, ou um objeto do tipo arquivo; utiliza virgula como delimitador default.

* read_table => Carrega dados delimitados de um arquivo, um URL ou um objeto do tipo arquivo; utiliza tabulação ('\t') como delimitador default.

* read_fwf => Lê dados em formato de coluna com tamanho fixo (isto é, sem delimitadores).

* read_clipboard => Versão de 'read_table' que lê dados da área de transferência(clipboard); é útil para converter tabelas de página web.

* read_excel => Lê dados tabulares de um arquivo Excel XLS ou XLSX.

* read_hdf => Lê arquivos HDF5 escritos pelo pandas.

* read_html => Lê todas as tabelas que se encontram no documento HTML especificado.

* read_json => Lê dados de uma representação em string JSON (JavaScript Object Notation, ou Notação de objetos JavaScript).

* read_msgpack => Lê dados codificados pelo pandas no formato binário MessagePack.

* read_pickle => Lê um objeto arbitrário armazenado no formato pickle de Python.

* read_sas => Lê um conjunto de dados SAS armazenado em um dos formatos personalizados do sistema SAS.

* read_sql => Lê o resultado de uma consulta SQL(usando o SQLAlchemy) na forma de um DataFrame do pandas.

* read_stata => Lê um conjunto de dados no formato de arquivo Stata.

* read_feather => Lê o formato de arquivo binário Feather.

Foi apresentada uma visão geral de como atual essas funções, que foram criadas com o objetivo de converter dados de texto em um DataFrame.Os argumentos opcionais dessas funções podem se enquadrar em algumas categorias:

* Indexação => Para tratar uma ou mais colunas como o DataFrame devolvido, e se os nomes das colunas devem ser obtidos do arquivo, do usuário ou de nenhum deles.

* Inferência de tipos e conversão de dados => Inclui as conversões de valores definidas pelo usuário e uma lista personalizada de marcadores de valores ausentes.

* Parsing de data e hora => Inclui recursos de combinação, entre eles, combinação de informações de data e hora espalhadas em várias colunas em uma única coluna no resultado.

* Iteração => Suporte para iteração em partes de arquivos bem grandes.

* Problemas com dados sujos => Pular linhas ou um rodapé, comentários ou outras pequenas informações como dados numéricos com vírgulas para separar milhares.

Algumas dessas funções, como 'pandas.read_csv', fazem inferência de tipos, pois os tipos de dados das colunas não fazem parte do formato. Isso significa que não precisa necessariamente especificar quais colunas são numéricas, inteiras, booleanas ou string. Outros formatos de dados, como HDF5, Feather e msgpack, têm os tipos de dados armazenados no formato.

Lidar com datas e outros tipos personalizados pode exigir esforço extra.

In [1]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Considerando que os dados estão delimitados por vírgula, pode-se usar 'read_csv' para lê-los em um DataFrame:

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Também poderíamos ter usado 'read_table' e especificado o delimitador:

In [4]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Um arquivo sem sempre terá uma linha de cabeçalho. Considere o arquivo a seguir:

In [5]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Para ler esse arquivo, temos duas opções. Podemos permitir que o pandas atribua nomes default para as colunas ou podemos, nós mesmos, especificar os nomes:

In [6]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['x', 'y', 'z', 'w', 'mensagem'])

,x,y,z,w,mensagem
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suponha que quiséssemos que a coluna "mensagem" fosse o índice do DataFrame devolvido. Podemos informar que queremos a coluna no índice 4 ou de nome 'mensagem' utilizando o argumento 'index_col'.

In [8]:
names = ['x', 'y', 'z', 'w', 'mensagem']

In [9]:
pd.read_csv('examples/ex2.csv', names=names, index_col='mensagem')

,x,y,z,w
mensagem,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


Caso queira compor um índice hierárquico a partir de várias colunas, passe uma lista de números ou de nomes de colunas.

In [10]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])

In [12]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

Em alguns casos, uma tabela pode não ter um delimitador fixo, usando espaço em branco ou outro padrão para separar os campos. Considere um arquivo-texto com a aparência a seguir:

In [13]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

Embora possa fazer algumas manipulações manualmente, os campos, nesse caso, estão separados por uma quantidade variável de espaços em branco. Em situações como essa, é possível passar uma expressão regular como um delimitador para 'read_table'. Nesse exemplo, podemos usar a expressão regular '\s+', de modo que teríamos:

In [14]:
resultado = pd.read_table('examples/ex3.txt', sep='\s+')
resultado

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [15]:
resultado

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


As funções de 'parser' têm muitos argumentos adicionais para ajudar a lidar com uma grande variedade de possíveis formatos de arquivos excepcionais (veja uma lista parcial na tabela 6.2). Por exemplo, podemos ignorar a primeira, a terceira e a quarta linhas de um arquivo usando 'skiprows':

In [16]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [17]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Lidar com valores ausentes é uma parte importante e, em geral, com certas nuances, no processo de parsing do arquivo. Dados ausentes geralmente não estão presentes(são strings vazias) ou então marcados com algum valor de sentinela. Por padrão, o pandas utiliza um conjunto de sentinelas que ocorrem usualmente, como 'NA' e 'NULL'

In [18]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [19]:
resultado = pd.read_csv('examples/ex5.csv')
resultado

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isnull(resultado)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


A opção 'na_values' pode aceitar uma lista ou um conjunto de strings a serem considerados como valores ausentes:

In [21]:
resultado = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
resultado

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Sentinelas NA diferentes podem ser especificadas para cada coluna em um dicionário:

In [22]:
sentinels = {'message': ['foo', 'NA'], 'something':['two']}

In [23]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


A tabela 6.2 lista algumas das opções usadas com frequência no 'pandas.read_csv' e no 'pandas.read_table'.

###### Tabela 6.2 -> Alguns argumentos das funções read_csv/read_table

Argumento

* Path => String que indica o local no sistema de arquivos, o URL ou um objeto do tipo arquivo.

* sep ou delimiter => Sequência de caracteres ou uma expressão regular a ser usada para separar campos em cada linha.

* header => Número da linha usada como nomes de colunas; o default é 0 (primeira linha), mas deverá ser 'None' se não houver linha de cabeçalho.

* index_col => Números ou nomes de colunas a serem usados como índice das linhas no resultado; pode ser um único nome/número  ou uma lista deles se for um índice hieráquico.

* names => Lista de nomes de colunas para o resultado, pode ser combinado com 'header=None'.

* skiprows => Número de linhas a serem ignoradas no início do arquivo ou lista de números de linhas(começando de 0) a serem ignoradas.

* na_values => Sequência de valores a serem substituídos por NA.

* comment => Caractere(s) para separar comentários no final das linhas.

* parse_dates => Tenta fazer parse de dados para 'datetime', o default é 'False'. Se for 'True', tentará fazer parse de todas as colunas. Caso contrário, poderá especificar uma lista de números ou nomes de colunas para o parse. Se o elemento da lista for uma tupla ou uma lista, combinará várias colunas e fará o parse para data (por exemplo, se a data/hora estiverem separadas em duas colunas).

* keep_date_col => Se as colunas forem reunidas para parse de data, mantém as colunas unidas; o default é False.

* converters => Dicionário contendo o número ou o nome de colunas mapeadas para funções (por exemplo, {'foo':f} aplica a função 'f' em todos os valores da coluna 'foo').

* dayfirst => Ao fazer parsing de datas potencialmente ambíguas, trata-as como estando no formato internacional (por exemplo, 7/6/2012 -> 7 de junho de 2012); o default é False.

* date_parser => Função a ser usada para parse de datas.

* nrows => Número de linhas a serem lidas no início do arquivo.

* iterator => Devolve um objeto 'TextParser' para ler o arquivo aos poucos.

* chumksize => Para iteração, é o tamanho das partes dos arquivos.

* skip_foler => Número de linhas a serem ignoradas no final do arquivo.

* verbose => Exibe várias informações de saída do parser, como o número de valores ausentes em colunas não numéricas.

* encoding => Codificação de texto para o Unicode (por exemplo, 'utf-8', para texto codificação em UTF-8).

* thousands => Separador de milhar (por exemplo, ',' ou '.')

#### Lendo arquivos-texto em partes

Ao processar arquivos bem grandes ou descobrir o conjunto certo de argumentoss para processar corretamente um arquivo grande, talvez queira ler somente uma pequena parte ou iterar por porções menores do arquivo.

Antes de observar um arquivo grande, vamos alterar as configurações do pandas a fim de que a exibição dos dados seja mais compacta:

In [24]:
# Alterar as configurações do pandas para a exibição dos dados seja mais compacta:
pd.options.display.max_rows = 10

In [25]:
# Agora temos:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


Se você quiser ler apenas uma quantidade pequenas de linhas (evitando ler o arquivo todo), especifique isso usando 'nrows':

In [26]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


Para ler um arquivo em partes, especifique uma quantidade de linhas para 'chunksize:'

In [27]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

O objeto 'TextParser' devolvido por read_csv permite iterar pelas partes do arquivo de acordo com o 'chunksize'. Por exemplo, podemos iterar por 'ex6.csv', agregando os contadores de valores na coluna 'key', assim:

In [28]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
    
tot = tot.sort_values(ascending=False)

/tmp/ipykernel_6298/625920012.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


In [29]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

'TextParser' também oferece um método 'get_chunk' que permite ler partes de tamanho arbitrário:

#### Escrevendo dados em formato-texto

Os dados também podem ser exportados para um formato com delimitador. Vamos considerar um dos arquivos CSV que lemos antes:

In [30]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Usando o método 'to_csv' de DataFrame, podemos escrever os dados separados por vírgula em um arquivo:


In [31]:
data.to_csv('examples/lfb_out.csv')

In [32]:
!cat examples/lfb_out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Outros delimitadores podem ser usados, é claro (escreveremos em sys.stdout para que o texto resultante seja exibido no console):

In [33]:
import sys

In [34]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Valores ausentes aparecem como strings vazias na saída. Pode representá-las com outro valor de sentinela:

In [35]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Sem outras opções especificadas, os rótulos tanto das linhas quanto das colunas são escritos. Ambos podem ser desativados:

In [36]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Também podemos escrever somente um subconjunto das colunas, em uma ordem de preferência:

In [37]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Uma Series também tem um método 'to_csv':

In [38]:
dates = pd.date_range('1/1/2000', periods=7)

In [39]:
import numpy as np
ts = pd.Series(np.arange(7), index=dates)
ts

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64

In [40]:
ts.to_csv('examples/lfb_tseries.csv')

In [41]:
!cat examples/lfb_tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


#### Trabalhando com formatos delimitados

É possível carregar a maioria dos formatos de dados tabulares de disco usando funções como 'pandas.read_table'. Em alguns casos, porém, um pouco de processamento manual talvez seja necessário. Não é incomum receber um arquivo com uma ou mais linhas malformadas que poderão confundir 'read_table'. Para demonstrar o uso das ferramentas básicas, considere um pequeno arquivo CSV:

In [42]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


Para qualquer arquivo com um único caractere como delimitador, podemos usar o módulo embutido csv de Python. Para usá-lo, passe qualquer arquivo aberto ou um objeto do tipo arquivo para 'csv.reader':

In [43]:
import csv
f = open('examples/ex7.csv')

reader = csv.reader(f)

In [44]:
reader

Iterar pelo reader como um arquivo produz tuplas de valores com qualquer caractere de aspas removido:

In [45]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


A partir daqui, nos cabe fazer a manipulação necessária para deixar os dados no formato em que precisar. Executaremos essa tarefa passo a passo. Inicialmente iremos ler o arquivo em uma lista de linhas:

In [46]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

Em seguida, separamos as linhas em linha de cabeçalho e linhas de dados:

In [47]:
header, values = lines[0], lines[1:]

Então podemos criar um dicionário de colunas de dados usando uma dictionary comprehension (abrangência de dicionário) e a expressão zip(*values), que faz a transposição das linhas para as colunas:

In [48]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

As possíveis opções (atributos de csv.Dialect) e o que elas fazem podem ser encontrados na tabela 6.3

###### Tabela 6.3 => Opções de dialetos de CSV

Argumento

* delimiter => String de um caractere para separar os campos; o default  é ','.

* lineterminator => Finalizador de linha para escrita; o default é '\r\n'. O reader ignora isso e reconhece finalizadores de linha em plataformas diferentes.

* quotechar => Caracteres de aspas para campos com caracteres especiais (como um delimitador); o default é '"'.

* quoting => Convenção para aspas. As opções incluem csv.QUOTE_ALL(aspas em todos os campos), csv_QUOTE_MINIMAL(somente campos com caracteres especiais, como o delimitador), csv.QUOTE_NONNUMERIC e csv.QUOTE_NONE(sem aspas)

* skipinitialspace => Ignora espaços em branco depois de cada delimitador; o default é False.

* doublequote => Como lidar com o caractere de aspas em um campo; se for True, é duplo.

* escapechar => String para escapar o delimitador se quoting estiver definido com csv.QUOTE_NONE; o default é desativado.

Para escrever arquivos com delimitadores manualmente, podemos usar o csv.writer. Ele aceita um objeto de arquivo aberto, com permissão para escrita, além das mesmas opções de dialeto e de formato de csv.reader.

#### Dados JSON:
    
O JSON (Abreviatura de JavaScript Object Notation, ou Notação de Objetos JavaScript) tornou-se um dos formatos padrões para envio de dados em requisições HTTP entre navegadores web e outras aplicações. É um formato de dados muito mais livre que um formato de texto tabular como o CSV.

In [49]:
obj = """
{"name": "Wes", 
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null, 
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, 
              {"name": "Katie", "age": 38, 
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

Os tipos básicos são objetos (dicionários), arrays(listas), strings, números, booleanos e nulls. Todas as chaves em um objeto devem ser strings. Há várias bibliotecas Python para ler e escrever dados JSON. Para converter uma string JSON em formato Python, utilize 'json.loads'

In [50]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

'json.dumps', por outro lado, converte um objeto Python de volta para JSON:

In [51]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

É possível passar uma lista de dicionários (que eram anteriormente objetos JSON)para o construtor de DataFrame e selecionar um subconjunto dos campos de dados:

In [52]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


'pandas.read_json' pode converter automaticamente conjuntos de dados JSON organizados de modo específico em uma Series ou um DataFrame. Por exemplo:

In [53]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


As opções default para pandas.read_json supõem que cada objeto no array JSON seja uma linha da tabela:

In [54]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


Para um exemplo mais amplo de leitura e manipulação de dados JSON(incluindo registros aninhados).

Se precisar exportar dados do pandas para o JSON, uma maneira é usar os métodos 'to_json' em Series e em DataFrame.

In [55]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [56]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


#### XML e HTML: Web Scraping
    
Python tem muitas bibliotecas para ler e escrever dados nos formatos HTML e XML presentes em todos os lugares. Exemplos incluem lxml (http://lxml.de). Embora o lxml, em geral, seja comparativamente muito mais rápido, as outras bibliotecas podem lidar melhor com arquivos HTML ou XML malformados.

O pandas tem uma função embutida 'read_html' que utiliza bibliotecas como lxml e Beautiful Soup para fazer parse automaticamente de tabelas de arquivos HTML em objetos DataFrame. Devemos instalar algumas bibliotecas adicionais usadas por 'read_html'.

In [57]:
#!pip install lxml

In [58]:
#!pip install beautifulsoup4 html5lib

A função 'pandas.read_html' tem uma série de opções, mas, por padrão, ela procura e tenta fazer parse de todos os dados tabulares contidos em tags <table>. O resultado é uma lista de objetos DataFrame:

In [59]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [60]:
len(tables)

1

In [61]:
failures = tables[0]

In [62]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Como 'failures' tem muitas colunas, o pandas insere um caractere de quebra de linha \.

A partir daqui poderíamos prosseguir fazendo uma limpeza e a análise de dados, por exemplo, calculando o número de falências bancárias por ano:

In [63]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [64]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Fazendo parse de XML com lxml.objectify

O XML(eXtensible Markup Language ou Linguagem de Marcação Extensível) é outro formato comum de dados estruturados que aceita dados hierárquicos e aninhados com metadados. 

Foi mostrado a função 'pandas.read_html', que utiliza o lxml ou o Beautiful Soup internamente para fazer parse de dados de HTML, XML e HTML, são estruturamente semelhantes, porém o XML é mais genérico.

O MTA (Metropolitan Transportation Authority) de Nova York publica uma série de dados sobre seus serviços de ônibus e de trem (http://www.mta.info/developers/download.html).

Nesse exemplo, observaremos os dados de desempenho, que estão contidos em um conjunto de arquivos XML. Cada serviço de trem ou de ônibus tem um arquivo diferente (como Performance_MNR.xml) para o Metro-North Rail-road):

Usando 'lxml.objectify', fazemos o parse do arquivo e obtemos uma referência para o nó raiz do arquivo XML com 'getroot':

In [65]:
from lxml import objectify

path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

'root.INDICATOR' devolve um gerador que produz cada elemento XML<INDICATOR>. 

Para cada registro, podemos preencher um dicionário de nomes de tags(como YTD_ACTUAL) para valores de dados (exluíndo algumas tags):

In [66]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data ={}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

Por fim, converteremos essa lista de dicionários em um DataFrame:

In [67]:
perf = pd.DataFrame(data)
perf

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8
...,...,...,...,...,...,...,...,...,...,...,...,...
643,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,8,Service Indicators,M,%,97.0,,97.0,
644,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,9,Service Indicators,M,%,97.0,,97.0,
645,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,10,Service Indicators,M,%,97.0,,97.0,
646,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,11,Service Indicators,M,%,97.0,,97.0,


In [68]:
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


Dados XML podem ser tornar muito mais complexos que os dados desse exemplo. Cada tag pode ter metadados também. Considerando uma tag de link HTML, que também é um XML válido:

In [69]:
from io import StringIO

tag = '<a href="http://www.goggle.com.br">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

Agora podemos ter acesso a qualquer um dos campos (como href) da tag ou do texto de link:

In [70]:
root

<Element a at 0x7f84ae02a980>

In [71]:
root.get('href')

'http://www.goggle.com.br'

In [72]:
root.text

'Google'

### 6.2 Formatos de dados binários

Uma das formas mais simples de armazenar dados (também conhecida como serialização) de modo eficiente em formato binário é usando a serialização embutida 'pickle' do Python. Todos os objetos do pandas têm um método 'to_pickle' que escreve dados em disco em formato pickle:

In [73]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [74]:
frame.to_pickle('examples/lfb_frame_pickle')

Podemos ler qualquer objeto armazenado em um arquivo em formato 'pickle' utilizando diretamente a função embutida pickle ou, de modo mais conveniente ainda, usando 'pandas.read_pickle':

In [75]:
pd.read_pickle('examples/lfb_frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


O pandas tem suporte incluído para dois outros formatos de dados binários: HDF5 e MessagePack. 

Outros formatos de armazenagem para dados do pandas ou do NumPy incluem:

* bcolz (http://bcolz.blosc.org) => Um formato binário passível de compactação, orientado a colunas, baseado na biblioteca de compactação Blosc:

* Feather (http://github.com/wesm/feather) => Um formato de arquivo para várias linguagens, orientado a colunas.

#### Usando o formato HDF5

O HDF5 é um formato de arquivo bem-visto, cujo o propósito é armazenar grandes quantidades de dados científicos em arrays. Está disponível na forma de uma biblioteca C, e tem interfaces disponíveis em várias outras linguagens, incluindo Java, Julia, MatLab e Python. O 'HDF' em HDF5 quer dizer 'Hierarchical Data Format (Formato de Dados Hierárquico)'. Cada arquivo HDF5 é capaz de armazenar vários conjuntos de dados e pode aceitar metadados. 

Se comparado com formatos mais simples, o HDF5 aceita compactação durante a execução, com uma variedade de modos de compactação, permitindo que dados com padrões repetidos sejam armazenados de modo mais eficiente. O HDF5 pode ser uma boa opção para trabalhar com conjuntos bem grandes de dados que não caibam na memória, pois poderá ler e escrever pequenas seções de arrays muito maiores, de modo eficaz.

Embora seja possível acessar diretamente arquivos HDF5 usando as bibliotecas PyTables ou h5py, o pandas oferece uma interface de alto nível que simplifica o armazenamento de objetos Series e DataFrame. A classe 'HDFStore' funciona como um dicionário e cuida dos detalhes de baixo nível:

In [76]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [77]:
store = pd.HDFStore('mydata.h5')

In [78]:
store['obj1'] = frame

In [79]:
store['obj1_col'] = frame['a']

In [80]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

Objetos contidos no arquivo HDF5 podem então ser recuperados com a mesma API do tipo dicionário:

In [81]:
store['obj1']

,a
0,-0.271355
1,0.665841
2,-1.678398
3,0.153741
4,-0.854661
...,...
95,-1.088949
96,-0.159183
97,-0.644001
98,-0.547388


'HDFStore' aceita dois esquemas de armazenagem: 'fixed' e 'table'. O último em geral é mais lento, porém aceita operações de consulta usando uma sintaxe eespecial:

In [82]:
store.put('obj2', frame, format='table')

In [83]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-0.491188
11,-0.414392
12,-1.336540
13,1.161562
14,-0.502840
15,0.554463


O put é uma versão explícita do método store['obj2'] = frame, mas permite definir outras opções como o formato de armazenagem:
    
A função 'pandas.read_hdf' oferece um atalho para essas ferramentas:

In [84]:
frame.to_hdf('mydata.h5', 'obj3', format = 'table')

Se trabalha com grandes quantidades de dados localmente, incentivaria a explorar o PyTables e o h5py para ver como eles podem atender às suas necessidades. Como muitos problemas de análise de dados são limitados por E/S (são I/O-bound em vez de CPU-bound), usar uma ferramenta como HDF5 pode agilizar significamente as suas aplicações.

#### Lendo arquivos do Microsoft Excel

O pandas também oferece suporte para ler dados tabulares armazenados em arquivos do Excel 2003 usando a classe 'ExcelFile' ou a função 'pandas.read_excel'. Internamente essas ferramentas utilizam os pacotes add-on 'xlrd' e 'openpyxl' para ler arquivos XLS e XLSX, respectivamente 

Para usar 'ExcelFile', crie uma instância passando um path para um arquivo xls ou xlsx:

In [85]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
xlsx

Dados armazenados em uma planilha poderão então ser lidos em um DataFrame utilizando parse:

In [86]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


Se estiver lendo várias planilhas de um arquivo, será mais rápido criar o 'ExcelFile', mas também pode simplesmente passar o nome do arquivo para 'pandas.read_excel':

In [87]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


Para escrever dados do pandas em formato Excel, deve inicialmente criar um ExcelWriter e então escrever os dados, depois usando o método 'to_excel' de objeto do pandas:

In [88]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [89]:
frame.to_excel(writer, 'Sheet1')

In [90]:
writer.save()

In [91]:
writer

In [92]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


Um path de arquivo também pode ser passado para 'to_excel', evitando o ExcelWriter:

In [93]:
frame.to_excel('examples/ex2a.xlsx')

### 6.3 Interagindo com APIs web

Muitos sites têm APIs públicas que oferecem feeds de dados usando JSON ou outro formato. Há várias maneiras de acessar essas APIs a partir do Python, um método fácil de usar, é lançar mão do pacote requests (http://docs.python-requests.org).

Para encontrar os últimos 30 problemas do pandas no GitHub, podemos fazer uma requisição HTTP get usando a biblioteca add-on 'requests':

In [94]:
import requests

In [95]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [96]:
resp = requests.get(url)

In [97]:
resp

<Response [200]>

O método JSON do objeto Response devolverá um dicionário contendo o parse dos dados JSON em objetos Python nativos:

In [98]:
data = resp.json()

In [99]:
data[0]['title']

'API: rename DataFrame.applymap -> DataFrame.map'

In [100]:
data

[{'url': 'https://api.github.com/repos/pandas-dev/pandas/issues/52353',
  'repository_url': 'https://api.github.com/repos/pandas-dev/pandas',
  'labels_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/52353/labels{/name}',
  'comments_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/52353/comments',
  'events_url': 'https://api.github.com/repos/pandas-dev/pandas/issues/52353/events',
  'html_url': 'https://github.com/pandas-dev/pandas/issues/52353',
  'id': 1650674596,
  'node_id': 'I_kwDOAA0YD85iY0uk',
  'number': 52353,
  'title': 'API: rename DataFrame.applymap -> DataFrame.map',
  'user': {'login': 'topper-123',
   'id': 26364415,
   'node_id': 'MDQ6VXNlcjI2MzY0NDE1',
   'avatar_url': 'https://avatars.githubusercontent.com/u/26364415?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/topper-123',
   'html_url': 'https://github.com/topper-123',
   'followers_url': 'https://api.github.com/users/topper-123/followers',
   'following_url': 'https

Cada elemento em 'data' é um dicionário contendo todos os dados encontrados em uma página de problemas do GitHub (exceto os comentários). 

Podemos passar 'data' diretamente para DataFrame e extrair os campos de interesse:

In [101]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,52353,API: rename DataFrame.applymap -> DataFrame.map,[],open
1,52352,DOC: update documentation for YearBegin,[],open
2,52351,PERF: extract_array,[],open
3,52349,CI: Test development container,[],open
4,52348,PERF: DataFrame.values for pyarrow-backed nume...,"[{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...",open
...,...,...,...,...
25,52323,BUG: Interchange protocol fails with pyarrow b...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,52322,BUG: Unexpected behauvior interpolate limit wi...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
27,52321,BUG: AttributeError: 'ArrowTemporalProperties'...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,52319,BUG: IndexError when boolean indexing on pyarr...,"[{'id': 717120670, 'node_id': 'MDU6TGFiZWw3MTc...",open


Podemos criar lagumas interfaces de nível mais alto para APIs web comuns que devolvam objetos DataFrame a fim de facilitar a análise:

### 6.4 Interagindo com banco de dados

Em um ambiente de negócios, a maior parte dos dados talvez não esteja armazenada em arquivos-texto e nem em arquivos Excel. Bancos de dados relacionais em SQL(como SQL Server, PostgresSQL e MySQL) são amplamente usados, e muitos bancos de dados alternativos têm se tornado bem populares. A escolha do banco de dados em geral depende das necessidades de desempenho, de integridade dos dados e de escalabilidade de uma aplicação.

Carregar dados de SQL em um DataFrame é razoavelmente simples, e o pandas tem algumas funções para simplificar o processo. como exemplo, será criado um banco de dados SQLite usando o driver embutido 'sqlite3' do Python:

In [102]:
import sqlite3

In [103]:
query = """
    CREATE TABLE test
    (a VARCHAR(20), b VARCHAR(20),
     c REAL, d INTEGER
    );
"""

In [104]:
con = sqlite3.connect('mydata.sqlite')

In [105]:
con.execute(query)

In [106]:
con.commit()

em seguida, inserimos algumas linhas de dados:

In [107]:
data = [('Atlanta', 'Georgia', 1.25, 6), 
        ('Tallahasse', 'Floria', 2.6, 3), 
        ('Sacramento', 'California', 1.7, 5)]

In [108]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [109]:
con.executemany(stmt, data)

In [110]:
con.commit()

A maioria dis drivers de SQL de Python (PyODBC, psycopg2, MySQLdb, pymssql etc) devolve uma lista de tuplas ao selecionar dados de uma tabela:

In [111]:
cursor = con.execute('select * from test')

In [112]:
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahasse', 'Floria', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

Podemos passar a lista de tuplas para o construtor de DataFrame, mas também precisaremos dos nomes das colunas, contidos no atributo description do cursor:

In [113]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [114]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahasse,Floria,2.60,3
2,Sacramento,California,1.70,5


In [115]:
con.close()

Há uma boa dose de manipulação que seria melhor não repetir sempre que for consultar o banco de dados. O projeto SQLAlchemy (http://www.sqlalchemy.org/) é um kit de ferramentas SQL popular para Python, que abstrai muitas das diferenças comuns entre os bancos de dados SQL. 

O pandas tem uma função 'read_sql' que permite ler os dados facilmente de uma conexão SQLAlchemy genérica. Neste exemplo faremos a conexão com o mesmo banco de dados SQLite usando o SQLAlchemy e leremos dados da tabela criada anteriormente:

In [116]:
import sqlalchemy as sqla

In [117]:
db = sqla.create_engine('sqlite:///mydata.sqlite')

In [118]:
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahasse,Floria,2.60,3
2,Sacramento,California,1.70,5


### 6.5 Conclusão 

Ter acesso aos dados geralmente é o primeiro passo no processo de análise de dados. Neste capítulo foi visto uma série de ferramentas úteis que deverão ajudar a começar o trabalho.

Nos próximos capítulos, serão explorados com mais detalhes o tratamento e a visualização dos dado, a análise de séries temporais.